In [9]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st

# Load the data from the csv file, and parse the dates to datetime format
df_TAT = pd.read_csv("Daten CSV.csv", sep = ";", parse_dates=["Erfassung der Anforderung", "Probenverteilung", "Proben auf Strasse", 
    "Zentrifugation", "Analyten werden gemessen", "Resultate werden übermittel", 
    "Technische Validation", "Medizinische Validation", "Erstellen & Versenden Bericht"
]) 
df_TAT.head()
df_TAT.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Probennummer                   1 non-null      int64         
 1   Analyt                         1 non-null      object        
 2   Erfassung der Anforderung      1 non-null      datetime64[ns]
 3   Probenverteilung               1 non-null      datetime64[ns]
 4   Proben auf Strasse             1 non-null      datetime64[ns]
 5   Zentrifugation                 1 non-null      datetime64[ns]
 6   Analyten werden gemessen       1 non-null      datetime64[ns]
 7   Resultate werden übermittel    1 non-null      datetime64[ns]
 8   Technische Validation          1 non-null      datetime64[ns]
 9   Medizinische Validation        1 non-null      datetime64[ns]
 10  Erstellen & Versenden Bericht  1 non-null      datetime64[ns]
dtypes: datetime64[ns](9), i

In [12]:
# Add 7 minutes to "Zentrifugation" column to create "Zentrifugation Ende"
df_TAT['Zentrifugation Ende'] = df_TAT['Zentrifugation'] + pd.Timedelta(minutes=7)
# Rearrange columns to place "Zentrifugation Ende" after "Zentrifugation"
cols = df_TAT.columns.tolist()
zentrifugation_index = cols.index('Zentrifugation')
cols.insert(zentrifugation_index + 1, cols.pop(cols.index('Zentrifugation Ende')))
df_TAT = df_TAT[cols]
df_TAT.head()


,Probennummer,Analyt,Erfassung der Anforderung,Probenverteilung,Proben auf Strasse,Zentrifugation,Zentrifugation Ende,Analyten werden gemessen,Resultate werden übermittel,Technische Validation,Medizinische Validation,Erstellen & Versenden Bericht
0,2211334401,Troponin,2024-10-01 08:40:00,2024-10-01 13:30:00,2024-10-01 14:05:00,2024-10-01 14:18:00,2024-10-01 14:25:00,2024-10-01 14:27:00,2024-10-01 15:10:00,2024-10-01 15:28:00,2024-10-01 15:49:00,2024-10-01 16:20:00


In [10]:
phases = {
    "Präanalytische Phase": ["Erfassung der Anforderung", "Probenverteilung", "Proben auf Strasse","Zentrifugation", "Zentrifugation Ende"],
    "Analytische Phase": ["Analyten werden gemessen", "Resultate werden übermittelt"],
    "Postanalytische Phase": ["Technische Validation","Medizinische Validation", "Erstellen & Versenden Bericht"]
}

# Example of accessing the data structure
for phase, checkpoints in phases.items():
    print(f"{phase}:")
    for checkpoint in checkpoints:
        print(f"  - {checkpoint}")

Präanalytische Phase:
  - Erfassung der Anforderung
  - Probenverteilung
  - Proben auf Strasse
  - Zentrifugation
Analytische Phase:
  - Analyten werden gemessen
  - Resultate werden übermittel
Postanalytische Phase:
  - Technische Validation
  - Medizinische Validation
  - Erstellen & Versenden Bericht


In [13]:
for phase, checkpoints in phases.items():
    # Calculate the time difference between "Erfassung der Anforderung" and "Zentrifugation Ende"
    df_TAT[f"{phase}"] = df_TAT[checkpoints[-1]] - df_TAT[checkpoints[0]]

df_TAT.head()


,Probennummer,Analyt,Erfassung der Anforderung,Probenverteilung,Proben auf Strasse,Zentrifugation,Zentrifugation Ende,Analyten werden gemessen,Resultate werden übermittel,Technische Validation,Medizinische Validation,Erstellen & Versenden Bericht,Präanalytische Phase,Analytische Phase,Postanalytische Phase
0,2211334401,Troponin,2024-10-01 08:40:00,2024-10-01 13:30:00,2024-10-01 14:05:00,2024-10-01 14:18:00,2024-10-01 14:25:00,2024-10-01 14:27:00,2024-10-01 15:10:00,2024-10-01 15:28:00,2024-10-01 15:49:00,2024-10-01 16:20:00,0 days 05:38:00,0 days 00:43:00,0 days 00:52:00
